# Business Question 1:

Observing the landscape of diversity notably nationality and age diversity in workplace for registered companies under Companies House of United Kingdom. The observation groups companies into their respective sector and see how particular sector perform in regards of diversity. 

### Business Question 1: Code:

## Importing libraries

In [ ]:
# list of library for this project
!pip3 install requests
!pip install folium -q

import time
import requests
import pprint as pp
import datetime
from datetime import date 
from datetime import datetime
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import folium
import warnings
warnings.filterwarnings("ignore")

## Initialize starting function to retrieve data

In [ ]:
# function for calling the API to retrieve JSON

def call_api_with(url_extension):
    your_company_house_api_key ="a4d0afaa-b5e4-455a-80d4-4554c59c18cc"
            
    login_headers = {"Authorization":your_company_house_api_key}
    url = f"https://api.companieshouse.gov.uk/{url_extension}"
    # above: could be eg. https://api.companieshouse.gov.uk/search/companies?q=shop&items_per_page=1
    print(f'requesting: {url}') 
    # above, optional: printing, so that you see visually how many calls you are making
    res = requests.get(url, headers=login_headers) #, verify=False)
    return res.json()

In [ ]:
# search/load companies with specific number of companies to request

def search_for_companies(number_of_companies = 100):
    # for simplicity round up the number of returned companies to the nearest hundred. eg. 130 becomes 200
    page_size = 20
    number_of_pages = math.ceil(number_of_companies / page_size) # round up
    companies = []
    for page_index in range(0, number_of_pages):
        url = f"advanced-search/companies?items_per_page={page_size}&start_index={page_index*page_size}"
        companies += call_api_with(url).get('items', [])
    return pd.json_normalize(companies)

In [ ]:
# get all persons with significant control from company which contains specific keyword

def all_persons_in_company(company_number):
    url = f"company/{company_number}/officers"
    return call_api_with(url).get('items', [])


In [ ]:
# get set of company numbers from companies

def select_companies(input_list, how_many = 1000):
    length = how_many - 1
    result = pd.DataFrame(input_list)['company_number'][:length]
    return result

## Retrieve detailed companies dataset 

In [ ]:
# retrieve numbers of company and save it as variable
companies_dataset = search_for_companies(1000)

In [ ]:
# retrieve company numbers from companies_dataset
company_numbers = select_companies(companies_dataset, 1000)

Data Cleaning on SIC Codes information:

In [ ]:
# delete row that has no sic codes
companies_dataset = companies_dataset.dropna(subset='sic_codes')

# only pick one sic code from every company having more than one sic code
companies_dataset['sic_codes'] = companies_dataset['sic_codes'].apply(lambda x: x[0]) 

# do not pick sic code = 99999 as it represents dormant companies
companies_dataset = companies_dataset[companies_dataset['sic_codes'] != '99999']

In [ ]:
companies_dataset

Table above shows companies dataset that has been cleaned

## Analyze diversity based on nationalities for each sic code

In [ ]:
# retrieve unique sic from all of companies in our detailed dataset
unique_sic = companies_dataset.sic_codes.unique()
unique_sic

In [ ]:
# function to get overall average of non-british employees proportion for each sic code 

def get_diversity_nation(sic_list):
    global sic_diversity_nation
    sic_diversity_nation = pd.DataFrame(columns=['sic_code','percentage'])
    
    #loop procedure to retrieve data for each sic code
    for sic in sic_list:
        company_numbers = select_companies(companies_dataset[companies_dataset['sic_codes']== sic], 1000)
        company_numbers = pd.DataFrame(company_numbers).reset_index(drop=True)
        
        new_data = pd.DataFrame()
        for company_no in company_numbers['company_number']:
            added_data = pd.json_normalize(all_persons_in_company(str(company_no)))
            new_data = pd.concat([new_data,added_data])
        new_df = pd.DataFrame([[sic,1-(len(new_data[new_data['nationality'].isin(['British','English','Welsh'])])/len(new_data)),len(new_data)]], columns=['sic_code','percentage','total_officers'])
        sic_diversity_nation = pd.concat([sic_diversity_nation,new_df])
        sic_diversity_nation.sort_values(by='percentage', ascending=False, inplace = True)
    
    return 
            
        

#### Codes below contain data retrieving and storing on loop due to API data retrieving limitation

In [ ]:
# get age diversity information from sic index 0 to 49
get_diversity_nation(unique_sic[:50])

# save retrieved data to new dataframe
sic_diversity_nation_all = pd.DataFrame()
sic_diversity_nation_all = pd.concat([sic_diversity_nation_all, sic_diversity_nation])

In [ ]:
time.sleep(180)
# get age diversity information from sic index 50 to 79
get_diversity_nation(unique_sic[50:80])

# append the list
sic_diversity_nation_all = pd.concat([sic_diversity_nation_all, sic_diversity_nation])

In [ ]:
time.sleep(180)
# get age diversity information from sic index 80 to 129
get_diversity_nation(unique_sic[80:130])

# append the list
sic_diversity_nation_all = pd.concat([sic_diversity_nation_all, sic_diversity_nation])

In [ ]:
time.sleep(180)
# get age diversity information from sic index 130 to 179
get_diversity_nation(unique_sic[130:180])

# append the list
sic_diversity_nation_all = pd.concat([sic_diversity_nation_all, sic_diversity_nation])

In [ ]:
# get age diversity information from sic index 180 to end
get_diversity_nation(unique_sic[180:])

# append the list
sic_diversity_nation_all = pd.concat([sic_diversity_nation_all, sic_diversity_nation])

In [ ]:
# function to convert 'overall average of non-british employees proportion for each sic code' to new variable called 'diversity score'
# 'diversity score' range from 0.5 to 1.0. 
# Smaller means proportion of british and non-british employees are highly unbalaned, and higher means well-balanced
# 'diversity score' = 1.0 means the proportion of non-british employee and british employees are 50:50

def nationality_diversity_score(dataset):
    dataset['diversity_score'] = dataset['percentage'].apply(lambda x: 1-abs(0.5-x))
    dataset.sort_values(by='diversity_score', ascending=False, inplace = True)
    return dataset

In [ ]:
sic_diversity_nation_all = nationality_diversity_score(sic_diversity_nation_all).reset_index(drop=True)
sic_diversity_nation_all

From table above, diversity score = 1 represent precisely balanced proportion of British and non-british employees. Meanwhile, 0.5 diversity score meaning a company has either all of their employees are british or non-british.

the 'percentage' column explain the proportion of non-british employee at a company

## Analyze diversity based on age

In [ ]:
def get_diversity_age(sic_list):
    global sic_diversity_age
    sic_diversity_age = pd.DataFrame(columns=['sic_code','standard_deviation'])
    
    #loop procedure to retrieve data for each sic code
    for sic in sic_list:
        company_numbers = select_companies(companies_dataset[companies_dataset['sic_codes']== sic], 1000)
        company_numbers = pd.DataFrame(company_numbers).reset_index(drop=True)
        
        # subloop to retrieve officers details and calculate the age when they starting to work at a company
        new_data = pd.DataFrame()
        for company_no in company_numbers['company_number']:
            added_data = pd.json_normalize(all_persons_in_company(str(company_no)))
            new_data = pd.concat([new_data,added_data])
        new_data['Appointed Year'] = pd.to_datetime(new_data['appointed_on']).dt.year
        new_data['Birth Year'] = new_data['date_of_birth.year']
        new_data['Age'] = new_data['Appointed Year'] - new_data['Birth Year']
        new_data.dropna(subset='Age', inplace=True)    
        
        # make a new dataframe containing diversity information for each sic code 
        new_df = pd.DataFrame([[sic,new_data.Age.std(),len(new_data)]], columns=['sic_code','standard_deviation','total_officers'])
        sic_diversity_age = pd.concat([sic_diversity_age,new_df])
        sic_diversity_age.sort_values(by='standard_deviation', ascending=False, inplace=True)
        sic_diversity_age.dropna(subset='standard_deviation', inplace=True)
    
    return 
            
        

#### Below codes contain data retrieving and storing on loop due to API data retrieving limitation

In [ ]:
# for n in range(0, len(unique_sic) // 50):
#     get_diversity_age(unique_sic[n * 50 : (n + 1) * 50])
#     time.sleep(1)

# get age diversity information from sic index 0 to 49
get_diversity_age(unique_sic[:50])

# save retrieved data to dataframe
sic_diversity_age_all = pd.DataFrame()
sic_diversity_age_all = pd.concat([sic_diversity_age_all, sic_diversity_age])


In [ ]:
time.sleep(180)
# get age diversity information from sic index 50 to 79
get_diversity_age(unique_sic[50:80])


# save and append the retrieved data
# sic_diversity_age_all = sic_diversity_age_all.append(sic_diversity_age)
sic_diversity_age_all = pd.concat([sic_diversity_age_all, sic_diversity_age])

In [ ]:
time.sleep(180)
# get age diversity information from sic index 80 to 129
get_diversity_age(unique_sic[80:130])

# save and append the retrieved data
sic_diversity_age_all = pd.concat([sic_diversity_age_all, sic_diversity_age])

In [ ]:
time.sleep(180)
# get age diversity information from sic index 130 to 180
get_diversity_age(unique_sic[130:180])

# save and append the retrieved data
sic_diversity_age_all = pd.concat([sic_diversity_age_all, sic_diversity_age])

In [ ]:
# get age diversity information from sic index 130 to 180
get_diversity_age(unique_sic[180:])

# save and append the retrieved data
sic_diversity_age_all = pd.concat([sic_diversity_age_all, sic_diversity_age])

In [ ]:
sic_diversity_age_all.reset_index(drop=True)

table above summarize the standard deviation of age for each sic code (business sectors)

## Visualize the analysis result of age and nationalities diversity

In [ ]:
def plot_hist(dataset, column):
    sns.histplot(dataset[column])

In [ ]:
plot_hist(sic_diversity_nation_all, 'diversity_score')

Histogram above explain how percentage of non-british employee of all companies being distributed

In [ ]:
plot_hist(sic_diversity_age_all, 'standard_deviation')

Histogram above explain how age standard deviation of all companies being distributed

In [ ]:
# function to convert parameters to classification class for better interpretation

def convert_to_class(dataset, diversity_type):
    dataset['class'] = 0
    if diversity_type == 'nationalities':
        dataset.loc[(dataset['percentage']>= 0.35) | (dataset['percentage']<= 0.65),'class'] = 'Good Diversity'
        dataset.loc[(dataset['percentage']> 0.65) | (dataset['percentage']< 0.35),'class'] = 'Moderate Diversity'
        dataset.loc[(dataset['percentage']>= 0.85) | (dataset['percentage']<= 0.15),'class'] = 'Less Diversity' 
    elif diversity_type == 'age':
        dataset.loc[(dataset['standard_deviation'] <= 5),'class'] = 'Less Diversity'
        dataset.loc[(dataset['standard_deviation'] > 5),'class'] = 'Moderate Diversity'
        dataset.loc[(dataset['standard_deviation'] > 10),'class'] = 'Good Diversity'
    return 

In [ ]:
convert_to_class(sic_diversity_nation_all, 'nationalities')
convert_to_class(sic_diversity_age_all, 'age')

In [ ]:
def plot_hist_class(dataset, title):
#     plt.figure(figsize=(8,5))
    sns.histplot(dataset['class'], stat='percent', label='total sic = '+str(len(dataset)))
    plt.title(title)
    plt.legend()

In [ ]:
figure_1 = plot_hist_class(sic_diversity_nation_all, 'Nationality Diversity')
figure_1

From graph above, it can be seen that most of business sectors stil do not have good diversity in their workplace's employee nationality

In [ ]:
figure_2 = plot_hist_class(sic_diversity_age_all, 'Age Diversity')
figure_2

From graph above, it can be seen that most of business sectors have been good when it comes to age diversity

In [ ]:
# retrieve 5 best and 5 worst business sector represented by sic code according to their parameter score
# best business sectors have highest 'diversity score' for diversity in nationalities, and otherwise
# best business sectors have highest 'standard deviation' for diversity in age, and otherwise

def best_and_worst(dataset, types):
    
    sic_decoder = pd.read_csv('/kaggle/input/companies-house-api-dataset/SIC.csv')
    sic_decoder['SIC Code'] = sic_decoder['SIC Code'].astype(str)
    sic_decoder.rename(columns={'SIC Code': 'sic_code'}, inplace=True)
    dataset = dataset.merge(sic_decoder, how='inner', on='sic_code')
    
    summary_table = pd.DataFrame()
    
    if types == 'nationalities': 
        summary_table['Best Sector Nationalities'] = dataset.Description.iloc[-5:].values
        summary_table['Worst Sector Nationalities'] = dataset.Description.iloc[:5].values
    else:
        summary_table['Best Sector Age'] = dataset.Description.iloc[-5:].values
        summary_table['Worst Sector Age'] = dataset.Description.iloc[:5].values
            
    return summary_table

In [ ]:
best_and_worst(sic_diversity_nation, 'nationalities')

In [ ]:
best_and_worst(sic_diversity_age, 'Age')

In [ ]:
combined_summary = best_and_worst(sic_diversity_nation, 'nationalities').join(best_and_worst(sic_diversity_age, 'Age'))
combined_summary

Combined dataframe above consist of best sector and worst sector for nationality and age diversity

### Business Question 1: Mini-report and visualisation:

Total words = 235

Promoting and delivering Equality, Diversity and Inclusivity (EDI) in the workplace is an essential aspect of good people management where every individual can feel safe, a sense of belonging and is empowered to achieve their full potential. This study aims to examine how far does UK-based companies implement EDI through observing how **nationality diversity** and **age diversity** differs for every business sector in overall represented by their sic code. 

The result shows on Figure 1 that 45% of total 216 different business sector still appear less diverse in nationality where number of British employees takes up large proportion overall, and vice-versa. On the other hand, Figure 2 shows how most of business sector already have good diversity in terms of age. 56% of total 178 different business sector recorded an age standard deviation more than 10 years. Different total business sector between nationality and age diversity analysis is due to data cleaning process of 1,000 different companies.

As this insight could be used to promote and push companies for EDI, it could also be utilized to inform public, notably jobseeker to assess what are best sectors that have good inclusivity of nationality and age. It can be seen from summary table below that 'Tax Consultancy' and 'Wholesale of pharmaceutical goods' are among best sectors for nationality diversity. Furthermore, we can see 'Repair of machinery' and 'Manufacture of beer' sectors provide good diversity in employee age.

In [ ]:
plt.subplots(1,2, figsize=(15,5))

plt.subplot(1,2,1)
plot_hist_class(sic_diversity_nation_all, 'Figure 1: Nationality Diversity')

plt.subplot(1,2,2)
plot_hist_class(sic_diversity_age_all, 'Figure 2: Age Diversity')



In [ ]:
combined_summary

# Business Question 2:

Evaluating potential location for IT consulting company's brach opening across United Kingdom. Perform analysis on dataset of dissolved companies to retrieve information of average age of dissolved companies in cities and dataset of active companies to retrieve information of potential market share in cities.

### Business Question 2: Code:

## Initializing function to retrieve data

In [ ]:
# function for calling the API to retrieve JSON
def call_api_with(url_extension):
    your_company_house_api_key ="a4d0afaa-b5e4-455a-80d4-4554c59c18cc"
            
    login_headers = {"Authorization":your_company_house_api_key}
    url = f"https://api.companieshouse.gov.uk/{url_extension}"
    # above: could be eg. https://api.companieshouse.gov.uk/search/companies?q=shop&items_per_page=1
    print(f'requesting: {url}') 
    # above, optional: printing, so that you see visually how many calls you are making
    res = requests.get(url, headers=login_headers) #, verify=False)
    return res.json()

In [ ]:
# search company only for company that is active
def search_for_companies_active(sic_code, number_of_companies = 2000,  company_status = 'active'):
    global active_companies
    # for simplicity round up the number of returned companies to the nearest hundred. eg. 130 becomes 200
    page_size = 20
    number_of_pages = math.ceil(number_of_companies / page_size) # round up
    active_companies = []
    for page_index in range(0, number_of_pages):
        url = f"advanced-search/companies?items_per_page={page_size}&start_index={page_index*page_size}&sic_codes={sic_code}&company_status={company_status}"
        active_companies += call_api_with(url).get('items', [])
    active_companies = pd.json_normalize(active_companies)
    return 
# &company_status={company_status1}&company_status={company_status2}

In [ ]:
# search company only for company that is dissolved
def search_for_companies_dissolved(sic_code,company_status = ['dissolved','liquidation'], number_of_companies = 100):
    global dissolved_companies
    # for simplicity round up the number of returned companies to the nearest hundred. eg. 130 becomes 200
    page_size = 20
    number_of_pages = math.ceil(number_of_companies / page_size) # round up
    dissolved_companies = []
    for page_index in range(0, number_of_pages):
        url = f"advanced-search/companies?items_per_page={page_size}&start_index={page_index*page_size}&sic_codes={sic_code}&company_status={company_status[0]}&company_status={company_status[1]}"
        dissolved_companies += call_api_with(url).get('items', [])
    dissolved_companies = pd.json_normalize(dissolved_companies)
    return 
# &company_status={company_status1}&company_status={company_status2}

## Get dataset for dissolved company 
The dataset will be used for later analysis about average company age of dissolved company

In [ ]:
# Get and filter only for company in Information and Technology Consultancy sector (sic code = 62020)

search_for_companies_dissolved('62020', number_of_companies = 2000)

In [ ]:
# banish data row that has no date_of_cessation or date_of_creation details

dissolved_companies.dropna(subset='date_of_cessation', inplace = True)
dissolved_companies.dropna(subset='date_of_creation', inplace = True)

In [ ]:
# function to calculate company age

def age_of_company(dataset):
    dataset['date_of_creation'] = pd.to_datetime(dataset['date_of_creation'], format='mixed')
    dataset['date_of_cessation'] = pd.to_datetime(dataset['date_of_cessation'], format='mixed')
    dataset['age_of_company_in_days'] =  dataset['date_of_cessation'] - dataset['date_of_creation']
    dataset.sort_values(by='age_of_company_in_days', inplace=True, ascending=False)
    
#     dataset['years'] = dataset['age_of_company_in_days'].dt.days // 365
#     dataset['months'] = (dataset['age_of_company_in_days'].dt.days - dataset['years'] *365) // 30
    years = dataset['age_of_company_in_days'].dt.days // 365
    months = (dataset['age_of_company_in_days'].dt.days - years *365) // 30
    dataset['age_of_company'] = 0
    for company in range(len(dataset)):
        dataset['age_of_company'].iloc[company] = str(years.iloc[company]) + " years, " + str(months.iloc[company]) + " months" 
    
    return dataset

In [ ]:
age_of_company(dissolved_companies).reset_index(drop=True)

Table above is dissolved companies table added with company age (sreation to cessation) being calculated.

## Analyze the average company age of dissolved company for each city

In [ ]:
# renaming column for better interpretability
dissolved_companies.rename(columns={"registered_office_address.locality": "city"}, inplace=True)

In [ ]:
# create new dataframe by grouping city and count companies for each city

dissolved_companies_per_city = pd.DataFrame(dissolved_companies['city'].value_counts().reset_index())


In [ ]:
dissolved_companies_per_city.rename(columns={'index':'city',"count": "count_companies"}, inplace=True)

In [ ]:
dissolved_companies_per_city

In [ ]:
# calculate the average company age using the prior main dataset of dissolved companies

dissolved_companies_per_city['average_company_age'] = 0
for index, city in enumerate(dissolved_companies_per_city['city']):
    dissolved_companies_per_city['average_company_age'].iloc[index] = dissolved_companies[dissolved_companies['city'] == city]['age_of_company_in_days'].dt.days.mean()

In [ ]:
# only choose city wherein there were more than five companies.
# This will avoid bias of overperforming/underperforming city with low number of company

dissolved_companies_per_city = dissolved_companies_per_city[dissolved_companies_per_city['count_companies'] > 5].reset_index(drop=True)

In [ ]:
# sort companies based on largest average company age

dissolved_companies_per_city.sort_values(by='average_company_age', ascending = False, inplace=True)

dissolved_companies_per_city

Table above shows information of best performing city with more than 5 companies previously dissolved and calculate the average company age of those companies. This gives sense of how good a city provide a sustainability for a company by giving insight of such age expectancy

## Get dataset for active company 
The dataset will be used for later analysis about current number of competitor companies and potential market provided

In [ ]:
# Get and filter only for company in Information and Technology Consultancy sector (sic code = 62020)

search_for_companies_active('62020', number_of_companies = 2000)

In [ ]:
active_companies

Table above depicts list of active IT consultancy firms

## Calculating potential market that is provided for each city

In [ ]:
# count the number of companies for each city
active_companies_per_city = pd.DataFrame(active_companies['registered_office_address.locality'].value_counts().reset_index())
active_companies_per_city

In [ ]:
# renaming column to get better clarity of column name interpretation
active_companies_per_city.rename(columns={'index':'city',"registered_office_address.locality": "city"}, inplace=True)

In [ ]:
# new csv file is being introduced
# this csv file contains population information for each city in United Kingdom, and also their respective latitude and longitude information

city_population = pd.read_csv('/kaggle/input/companies-house-api-dataset/gb_population.csv')
city_population

new dataset 'city_population' contains several information which we need for further analysis which are latitude, longitude and population

In [ ]:
active_companies_per_city

In [ ]:
# merge active_company_per_city dataset with city_population dataset

active_companies_per_city = active_companies_per_city.merge(city_population, how='left', on='city')

In [ ]:
#dropping row that hasno latitude and longitude informaton

active_companies_per_city.dropna(subset=['lat','lng'], inplace = True)


In [ ]:
# retrieve the potential market information

active_companies_per_city['potential_market'] = round(active_companies_per_city['population'] / active_companies_per_city['count'])
active_companies_per_city.sort_values(by='potential_market', ascending=False, inplace = True)

In [ ]:
active_companies_per_city[['city','potential_market']].head(10)

table above shows top 10 best performing city in terms of potential market.

Potential market per city is calculated with formula of: population / number of existing companies

### Visualizing 'average company age' from dissolved companies, and 'potential_market' from active_companies

In [ ]:
def top10_barplot(dataset, x, title):
    sns.barplot(x = x , y = "city", data = dataset[:10],edgecolor = "white", palette = "pastel")
    plt.title(title)

In [ ]:
plt.subplots(1,2 , figsize=(15,5))

plt.subplot(1,2,1)
top10_barplot(dissolved_companies_per_city, 'average_company_age', 'Top 10 Best City to start IT Consulting: \n Based on average age of dissolved company')
plt.subplot(1,2,2)
top10_barplot(active_companies_per_city, 'potential_market', 'Top 10 Best City to start IT Consulting: \n Based on potential market given current competitors')

plt.tight_layout()

## Plotting a map for best potential market

In [ ]:
# create map plotting function

def plot_to_map(dataset):
    m = folium.Map(location=[52.3555, -1.2743], zoom_start=6.3, tiles="OpenStreetMap")
    for city in range(10):
        folium.Marker([dataset['lat'].iloc[city], dataset['lng'].iloc[city]], popup='Potential Market: '+ str(dataset['potential_market'].iloc[city]) , tooltip=str(dataset['city'].iloc[city])).add_to(m)
    return m

In [ ]:
plot_to_map(active_companies_per_city)

### Business Question 2: Mini-report and visualisation:

Total words: 223

Edinburgh Consulting Group is a startup in Information Technology Consultancy sector. Embracing their major next step, they want to open new branches accross the United Kingdom. They utilize companies house of UK dataset to examine and assess **potential city location** in deciding in which cities do they want to open the new branches.

One parameter used in analysis is **average of company age** in cities that is calculaed from companies dataset only for already dissolved companies. The other parameter is **potential market** calculated from dividing city's population with current active companies as existing competitors, assuming that each companies has equal demand or market power and assuming how demand could be represented by population.

The analysis generates top 10 best cities to start/open IT consulting firm based on average age of dissolved companies (Figure 1) and based on potential market (Figure 2). From the graph figure 1, Essex is the best city with companies average age of 3,612 days or 9 years and 10 months. And from graph figure 2, Middlesbrough offers the best potential market demand with 174,700. These results are generated by processing dataset of 2,000 dissolved IT Consulting companies, and 2,000 active IT Consulting companies 

Cities' latitude and longitude are used to give better geographical understanding about top cities that performing best in terms of potential market depicted by map below.

In [ ]:
plt.subplots(1,2 , figsize=(15,5))

plt.subplot(1,2,1)
top10_barplot(dissolved_companies_per_city, 'average_company_age', 'Figure 1: Top 10 Best City to start IT Consulting: \n Based on average age of dissolved company')
plt.subplot(1,2,2)
top10_barplot(active_companies_per_city, 'potential_market', 'Figure 2: Top 10 Best City to start IT Consulting: \n Based on potential market given current competitors')

plt.tight_layout()

In [ ]:
plot_to_map(active_companies_per_city)